In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('https://raw.githubusercontent.com/dandynaufaldi/particle-swarm-optimized-clustering/master/seed.txt', sep='\t', header=None)
y = data[7]
data = data.drop([7], axis=1)
#data = data[1]
data = np.array(data).reshape(data.shape[0], 7)
scaler = MinMaxScaler()
data = scaler.fit_transform(data)  # normalize

In [ ]:
def BC(labels):
  sum = 0
  for i in range(len(labels)):
    for j in range(len(labels)):
      if labels[i] != labels[j]:
        sum += distance.euclidean(data[i], data[j])
  return sum

In [ ]:
def WC(labels):
  sum = 0
  for i in range(len(labels)):
    for j in range(len(labels)):
      if labels[i] == labels[j]:
        sum += distance.euclidean(data[i], data[j])
  return sum

In [ ]:
from sklearn.metrics import silhouette_score

def fit_func(labels):
  FF = (BC(labels)/WC(labels)) + silhouette_score(data, labels)
  return 1/FF

#KMeans

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3)
km.fit(data)
km_labels = km.predict(data)
print(km_labels)
print(fit_func(km_labels))

[2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2 2 2
 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 2 0 2 2 0 2 2 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 1 1]
0.19270100689710612


In [ ]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0

## Differential Evolution

In [ ]:
import random
import numpy
import time
#from solution import solution


# Differential Evolution (DE)
# mutation factor = [0.5, 2]
# crossover_ratio = [0,1]
def DE(objf, lb, ub, dim, PopSize, iters):

    mutation_factor = 0.5
    crossover_ratio = 0.7
    stopping_func = None

    # convert lb, ub to array
    if not isinstance(lb, list):
        lb = [lb for _ in range(dim)]
        ub = [ub for _ in range(dim)]

    # solution
    s = solution()

    s.best = float("inf")

    # initialize population
    population = []

    population_fitness = numpy.array([float("inf") for _ in range(PopSize)])

    for p in range(PopSize):
        sol = []
        for d in range(dim):
            d_val = random.randint(lb[d], ub[d])
            sol.append(d_val)

        population.append(sol)

    population = numpy.array(population)

    # calculate fitness for all the population
    for i in range(PopSize):
        fitness = objf(population[i, :])
        population_fitness[p] = fitness
        # s.func_evals += 1

        # is leader ?
        if fitness < s.best:
            s.best = fitness
            s.leader_solution = population[i, :]

    convergence_curve = numpy.zeros(iters)
    # start work
    print('DE is optimizing  "' + objf.__name__ + '"')

    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")

    t = 0
    while t < iters:
        # should i stop
        if stopping_func is not None and stopping_func(s.best, s.leader_solution, t):
            break

        # loop through population
        for i in range(PopSize):
            # 1. Mutation

            # select 3 random solution except current solution
            ids_except_current = [_ for _ in range(PopSize) if _ != i]
            id_1, id_2, id_3 = random.sample(ids_except_current, 3)

            mutant_sol = []
            for d in range(dim):
                d_val = population[id_1, d] + mutation_factor * (
                    population[id_2, d] - population[id_3, d]
                )

                # 2. Recombination
                rn = random.uniform(0, 1)
                if rn > crossover_ratio:
                    d_val = population[i, d]

                # add dimension value to the mutant solution
                mutant_sol.append(d_val)

            # 3. Replacement / Evaluation

            # clip new solution (mutant)
            mutant_sol = numpy.clip(mutant_sol, lb, ub)

            # calc fitness
            mutant_fitness = objf(mutant_sol)
            # s.func_evals += 1

            # replace if mutant_fitness is better
            if mutant_fitness < population_fitness[i]:
                population[i, :] = mutant_sol
                population_fitness[i] = mutant_fitness

                # update leader
                if mutant_fitness < s.best:
                    s.best = mutant_fitness
                    s.leader_solution = mutant_sol

        convergence_curve[t] = s.best
        if t % 1 == 0:
            print(
                ["At iteration " + str(t + 1) + " the best fitness is " + str(s.best)]
            )

        # increase iterations
        t = t + 1

        timerEnd = time.time()
        s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
        s.executionTime = timerEnd - timerStart
        s.convergence = convergence_curve
        s.optimizer = "DE"
        s.objfname = objf.__name__

    # return solution
    return s

In [ ]:
x = DE(objf = fit_func, lb = 0, ub = 2, dim = data.shape[0], PopSize = 10, iters = 1000)

DE is optimizing  "fit_func"
['At iteration 1 the best fitness is 0.2906892863865418']
['At iteration 2 the best fitness is 0.2751881873202973']
['At iteration 3 the best fitness is 0.2751881873202973']
['At iteration 4 the best fitness is 0.2751881873202973']
['At iteration 5 the best fitness is 0.2751881873202973']
['At iteration 6 the best fitness is 0.2751881873202973']
['At iteration 7 the best fitness is 0.2751881873202973']
['At iteration 8 the best fitness is 0.2751881873202973']
['At iteration 9 the best fitness is 0.2751881873202973']
['At iteration 10 the best fitness is 0.2751881873202973']
['At iteration 11 the best fitness is 0.2751881873202973']
['At iteration 12 the best fitness is 0.2751881873202973']
['At iteration 13 the best fitness is 0.2751881873202973']
['At iteration 14 the best fitness is 0.2751881873202973']
['At iteration 15 the best fitness is 0.2751881873202973']
['At iteration 16 the best fitness is 0.2751881873202973']
['At iteration 17 the best fitness i

#Grey Wolf Optimization

In [ ]:
import random
import numpy
import math
#from solution import solution
import time


def GWO(objf, lb, ub, dim, SearchAgents_no, Max_iter):

    # Max_iter=1000
    # lb=-100
    # ub=100
    # dim=30
    # SearchAgents_no=5

    # initialize alpha, beta, and delta_pos
    Alpha_pos = numpy.zeros(dim)
    Alpha_score = float("inf")

    Beta_pos = numpy.zeros(dim)
    Beta_score = float("inf")

    Delta_pos = numpy.zeros(dim)
    Delta_score = float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim

    # Initialize the positions of search agents
    Positions = numpy.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = (
            numpy.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
        )

    Convergence_curve = numpy.zeros(Max_iter)
    s = solution()

    # Loop counter
    print('GWO is optimizing  "' + objf.__name__ + '"')

    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0, Max_iter):
        for i in range(0, SearchAgents_no):

            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i, j] = numpy.clip(Positions[i, j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness = objf(Positions[i, :])

            # Update Alpha, Beta, and Delta
            if fitness < Alpha_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = Alpha_score  # Update beta
                Beta_pos = Alpha_pos.copy()
                Alpha_score = fitness
                # Update alpha
                Alpha_pos = Positions[i, :].copy()

            if fitness > Alpha_score and fitness < Beta_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = fitness  # Update beta
                Beta_pos = Positions[i, :].copy()

            if fitness > Alpha_score and fitness > Beta_score and fitness < Delta_score:
                Delta_score = fitness  # Update delta
                Delta_pos = Positions[i, :].copy()

        a = 2 - l * ((2) / Max_iter)
        # a decreases linearly fron 2 to 0

        # Update the Position of search agents including omegas
        for i in range(0, SearchAgents_no):
            for j in range(0, dim):

                r1 = random.random()  # r1 is a random number in [0,1]
                r2 = random.random()  # r2 is a random number in [0,1]

                A1 = 2 * a * r1 - a
                # Equation (3.3)
                C1 = 2 * r2
                # Equation (3.4)

                D_alpha = abs(C1 * Alpha_pos[j] - Positions[i, j])
                # Equation (3.5)-part 1
                X1 = Alpha_pos[j] - A1 * D_alpha
                # Equation (3.6)-part 1

                r1 = random.random()
                r2 = random.random()

                A2 = 2 * a * r1 - a
                # Equation (3.3)
                C2 = 2 * r2
                # Equation (3.4)

                D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 2
                X2 = Beta_pos[j] - A2 * D_beta
                # Equation (3.6)-part 2

                r1 = random.random()
                r2 = random.random()

                A3 = 2 * a * r1 - a
                # Equation (3.3)
                C3 = 2 * r2
                # Equation (3.4)

                D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 3
                X3 = Delta_pos[j] - A3 * D_delta
                # Equation (3.5)-part 3

                Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)

        Convergence_curve[l] = Alpha_score

        if l % 1 == 0:
            print(
                ["At iteration " + str(l) + " the best fitness is " + str(Alpha_score)]
            )

    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = Convergence_curve
    s.optimizer = "GWO"
    s.objfname = objf.__name__

    return s

In [ ]:
x = GWO(objf = fit_func, lb = 0, ub = 1, dim = 4, SearchAgents_no = 120, Max_iter = 1500)

GWO is optimizing  "inertia"
['At iteration 0 the best fitness is 12.402697746042264']
['At iteration 1 the best fitness is 12.402697746042264']
['At iteration 2 the best fitness is 12.402697746042264']
['At iteration 3 the best fitness is 12.381792885731166']
['At iteration 4 the best fitness is 12.213079135469346']


KeyboardInterrupt: ignored